In [1]:
import math

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

In [250]:
img_path = "/Users/fathe/Downloads/output/srp_GX010027_preprocessed-oio-5.png"

In [251]:
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

In [263]:
plt.figure(figsize=(10, 5))
plt.imshow(image[:-image.shape[0]//4, :], cmap='gray')
plt.show()

In [264]:
image = image[:-image.shape[0]//4, :]

stripe_width = 50
shift = 50
sizes = []
unusable = 0

for i in range(0, image.shape[1]-stripe_width, shift):
    # Extract vertical stripes
    stripe = image[:, i:stripe_width+i]
    
    # Sum pixel intensities along the x-axis
    stripe_sum = np.sum(stripe, axis=1)
    
    stripe_sum = gaussian_filter1d(stripe_sum, sigma=7)
    
    distance = 200
    # Detect peaks
    peaks, _ = find_peaks(stripe_sum, height=4500, prominence=2000, width=15, rel_height=0.2, distance=distance)

    if len(peaks) == 2 or len(peaks) == 3:
        size = (peaks[-1] - peaks[0])/(len(peaks)-1)
        if abs(size-275) < 10:
            sizes.append(size)
        else:
            sizes.append(sizes[-1])
            unusable += 1
    else: 
        sizes.append(sizes[-1])
        unusable += 1
            
        # # movements.append(movements[-1])
        # # Plot the results
        # plt.figure(figsize=(10, 5))
        # plt.plot(stripe_sum, label="Stripe")
        # plt.title(f"Summed Pixel Intensities for frame {i}")
        # plt.xlabel("Vertical Position (pixels)")
        # plt.ylabel("Summed Intensity")
        # plt.legend()
        # 
        # # Display detected peaks
        # print("Peaks in stripe:", peaks)
        # print("Their sum values:", stripe_sum[peaks])
        # plt.show()
        # raise NotImplemented
    
print(unusable)

In [265]:
len(sizes), image.shape[1], max(sizes), min(sizes), np.argmax(sizes), np.argmin(sizes), np.median(sizes)

In [266]:
# Plot cumulative movements
plt.figure(figsize=(10, 5))
plt.plot(sizes, marker='o', linestyle='-', color='blue', label="Cumulative Movement")
plt.title("Cumulative Movements Over Stripes")
plt.xlabel("Stripe Index")
plt.ylabel("Cumulative Movement")
plt.legend()
plt.grid(True)
plt.show()

In [267]:
# image = image[image.shape[0]//4:, :]
image_stacked = np.hstack((image,) * 1)


i = 124 * shift
# Extract vertical stripes
stripe = image_stacked[:, i:stripe_width+i]

# Sum pixel intensities along the x-axis
stripe_sum = np.sum(stripe, axis=1)

stripe_sum = gaussian_filter1d(stripe_sum, sigma=7)

distance = 220
# Detect peaks
peaks, _ = find_peaks(stripe_sum, height=4500, prominence=2000, width=15, rel_height=0.2, distance=distance)
      
print((peaks[-1]-peaks[0])/(len(peaks)-1))
plt.figure(figsize=(10, 5))
plt.plot(stripe_sum, label="Stripe")
plt.title(f"Summed Pixel Intensities for frame {i}")
plt.xlabel("Vertical Position (pixels)")
plt.ylabel("Summed Intensity")
plt.legend()

# Display detected peaks
print("Peaks in stripe:", peaks)
print("Their sum values:", stripe_sum[peaks])
plt.show()

In [268]:
def sinusoid(x, A, B, C, D):
    return A * np.sin(B * x + C) + D

In [269]:
max(sizes), min(sizes)

In [270]:
x = np.arange(len(sizes))
initial_guesses = [(max(sizes) - min(sizes))/2, 0.0442477788, 0, 275]
lower_bounds = [0, 0.044246, -100, 260]  # Set lower bounds
upper_bounds = [(max(sizes) - min(sizes)), 0.044248, +100, 290]  # Set upper bounds
params, _ = curve_fit(sinusoid, x, sizes, p0=initial_guesses, bounds=(lower_bounds, upper_bounds))

In [271]:
params

In [272]:
A, B, C, D= params

# Generate fitted rotated sinusoid
fitted_sinusoid = sinusoid(x, A, B, C, D)
relative_stretch = (max(fitted_sinusoid) / (min(fitted_sinusoid)/100)) - 100
print(relative_stretch)
# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(x, sizes, label="Cumulative Movements")
plt.plot(x, fitted_sinusoid, label="Fitted Rotated Sinusoid", linestyle="--")
plt.legend()
plt.xlabel("Stripe Index")
plt.ylabel("Movement")
plt.title("Fitting Rotated Sinusoidal Model to Cumulative Movements")
plt.grid(True)
plt.show()

In [274]:
results = [1.5798657685579656, 1.6232253969969577, 0.7620101168199795, 1.1827585976205057, 0.09770879958135481, 1.0114873071663482]

In [275]:
np.mean(results), np.std(results)

In [2]:
img_path = "/Users/fathe/Downloads/output/pre_calibration.png"
img1 = cv2.imread(img_path)
img_path = "/Users/fathe/Downloads/output/post_calibration.png"
img2 = cv2.imread(img_path)
frames = [img1, img2]

In [5]:
columns = 1
rows = math.ceil(len(frames) / columns)
plt.figure(figsize=(12, rows*5))
titles = ["Original frame", "Rectified frame"]
for i, (frame, title) in enumerate(zip(frames, titles)):
    ax = plt.subplot(rows, columns, i+1)
    # ax.imshow(frame[detail_y[0]:detail_y[1], detail_x[0]:detail_x[1],0], cmap="gray")
    ax.imshow(frame, cmap='gray')
    ax.set_title(title)
plt.tight_layout()
plt.savefig("/Users/fathe/Downloads/rectification_example.png")
plt.show()
plt.close()